# 이미지의 분할과 객체 검출

### 그랩컷
- 그래프 컷 기반 영역 분할 알고리즘
- 영상의 픽셀을 그래프 정점으로 간주하고, 픽셀들을 두 개의 그룹으로 나누는 최적의 컷을 찾는 방식

### 니모 이미지 분할 예제

In [2]:
import sys
import numpy as np
import cv2

src = cv2.imread("../data/nemo.jpg")
if src is None:
    print('Image load failed!')
    sys.exit()

rc = cv2.selectROI(src)

mask = np.zeros(src.shape[:2], np.uint8)

cv2.grabCut(src, mask, rc, None, None, 5, cv2.GC_INIT_WITH_RECT)

mask_fg = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')  # 전경만 남김
mask_bg = np.where((mask == 1) | (mask == 3), 0, 1).astype('uint8')  # 배경만 남김

cv2.imshow('mask_fg', mask_fg * 255)
cv2.imshow('mask_bg', mask_bg * 255)

dst_fg = src * mask_fg[:, :, np.newaxis]  # 전경 영상
dst_bg = src * mask_bg[:, :, np.newaxis]  # 배경 영상

cv2.imshow('dst_fg', dst_fg)
cv2.imshow('dst_bg', dst_bg)

cv2.waitKey()
cv2.destroyAllWindows()

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


### 메시 이미지 분할

In [11]:
import sys
import numpy as np
import cv2

# 1. 입력 영상 불러오기
src = cv2.imread("../data/messi5.jpg")

if src is None:
    print('Image load failed!')
    sys.exit()

# 2. 마스크 및 모델 초기화
mask = np.zeros(src.shape[:2], np.uint8)              # 마스크
bgdModel = np.zeros((1, 65), np.float64)              # 배경 모델
fgdModel = np.zeros((1, 65), np.float64)              # 전경 모델

# 3. 사각형 지정 및 GrabCut 초기 실행 (사각형 기반)
rc = cv2.selectROI(src)
cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)

# 4. 초기 마스크 결과 시각화
mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')  # 0,2 = 배경
dst = src * mask2[:, :, np.newaxis]  # 전경만 출력

cv2.imshow('dst', dst)

# ----------------------------------------------------
# 5. 마우스 이벤트 등록 (전경/배경 브러시)
def on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)           # 파랑 점 (전경 표시)
        cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)           # 확실한 전경으로 마스크 표시
        cv2.imshow('dst', dst)

    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)       # 전경 드래그
            cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)
            cv2.imshow('dst', dst)
        elif flags & cv2.EVENT_FLAG_SHIFTKEY:
            cv2.circle(dst, (x, y), 3, (0, 0, 255), -1)       # 빨강 점 (배경 표시)
            cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)       # 확실한 배경으로 마스크 표시
            cv2.imshow('dst', dst)

cv2.setMouseCallback('dst', on_mouse)

# ----------------------------------------------------
# 6. 키보드 이벤트 (ENTER로 GrabCut 재실행, ESC로 종료)
while True:
    key = cv2.waitKey()

    if key == 13:  # ENTER 키 입력 시 → 사용자 수정 기반 GrabCut 재실행
        cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK)
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        dst = src * mask2[:, :, np.newaxis]
        cv2.imshow('dst', dst)

    elif key == 27:  # ESC 키 → 종료
        break

cv2.destroyAllWindows()

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


### 모양비교함수

- 스페이드 모양과 가장 비슷한 것 추출하는 예제

In [5]:
import sys
import numpy as np
import cv2

# 1. 영상 불러오기
obj = cv2.imread('../data/spades.jpg', cv2.IMREAD_GRAYSCALE)   # 비교 대상(스페이드)
src = cv2.imread('../data/symbols.jpg', cv2.IMREAD_GRAYSCALE)  # 여러 도형이 있는 입력 영상

if src is None or obj is None:
    print('Image load failed!')
    sys.exit()

# 2. 객체 영상 외곽선 검출
_, obj_bin = cv2.threshold(obj, 128, 255, cv2.THRESH_BINARY_INV)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
obj_pts = obj_contours[0]  # 비교 기준 외곽선

# 3. 입력 영상 외곽선 검출
_, src_bin = cv2.threshold(src, 128, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# 4. 결과 영상 준비 (컬러로 변환)
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

# 5. 입력 영상의 각 도형에 대해 반복 비교
for pts in contours:
    if cv2.contourArea(pts) < 1000:  # 너무 작은 노이즈 제거
        continue

    rc = cv2.boundingRect(pts)
    cv2.rectangle(dst, rc, (255, 0, 0), 1)  # 객체 위치 표시

    # 6. 외곽선 유사도 비교 (값이 작을수록 비슷함)
    dist = cv2.matchShapes(obj_pts, pts, cv2.CONTOURS_MATCH_I3, 0)

    # 7. 유사도 점수 표시
    cv2.putText(dst, str(round(dist, 4)), (rc[0], rc[1] - 3),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, cv2.LINE_AA)

    # 8. 기준보다 충분히 유사한 경우 빨간색 박스로 강조
    if dist < 0.1:
        cv2.rectangle(dst, rc, (0, 0, 255), 2)

# 9. 결과 출력
cv2.imshow('obj', obj)
cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 템플릿 매칭
- 입력 영상에서 (작은 크기의) 템플릿 영상과 일치하는 부분을 찾는 기법
- 템플릿 : 찾을 대상이 되는 작은 영상. 패치(patch)

In [25]:
import sys
import numpy as np
import cv2

# 입력 영상과 템플릿 영상 불러오기
src = cv2.imread('../data/circuit.bmp', cv2.IMREAD_GRAYSCALE)
templ = cv2.imread('../data/crystal.bmp', cv2.IMREAD_GRAYSCALE)

if src is None or templ is None:
    print('Image load failed!')
    sys.exit()

# 밝기 증가 + 가우시안 잡음 추가
noise = np.zeros(src.shape, np.int32)
cv2.randn(noise, 50, 10)  # 평균 50, 표준편차 10
src = cv2.add(src, noise, dtype=cv2.CV_8UC3)

# 템플릿 매칭
res = cv2.matchTemplate(src, templ, cv2.TM_CCOEFF_NORMED)

# 결과 정규화 (0~255 사이로)
res_norm = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

# 최댓값 및 위치 구하기
_, maxv, _, maxloc = cv2.minMaxLoc(res)
print('maxv:', maxv)
print('maxloc:', maxloc)

# 결과 영상에 빨간 사각형 그리기
th, tw = templ.shape[:2]
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.rectangle(dst, maxloc, (maxloc[0] + tw, maxloc[1] + th), (0, 0, 255), 2)

# 결과 영상 출력
cv2.imshow('src', src)
cv2.imshow('crystal', templ)
cv2.imshow('res_norm', res_norm)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

maxv: 0.9789559841156006
maxloc: (558, 323)


### 인식.
- 아래는 숫자 인식 예제

In [35]:
import sys
import numpy as np
import cv2

def load_digits():
    img_digits = []
    for i in range(10):
        filename = '../data/digits/digit{}.bmp'.format(i)
        img_digits.append(cv2.imread(filename, cv2.IMREAD_GRAYSCALE))
        if img_digits[i] is None:
            return None
    return img_digits

def find_digit(img, img_digits):
    max_idx = -1
    max_ccoeff = -1
    # 최대 NCC 찾기
    for i in range(10):
        img = cv2.resize(img, (100, 150))
        res = cv2.matchTemplate(img, img_digits[i], cv2.TM_CCOEFF_NORMED)
        if res[0, 0] > max_ccoeff:
            max_idx = i
            max_ccoeff = res[0, 0]
    return max_idx

def main():
    # 입력 영상 불러오기
    src = cv2.imread('../data/digits_print.bmp')
    if src is None:
        print('Image load failed!')
        return

    # 100x150 숫자 영상 불러오기
    img_digits = load_digits()
    # list of ndarray
    if img_digits is None:
        print('Digit image load failed!')
        return

    # 입력 영상 이진화 & 레이블링
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(src_bin)

    # 숫자 인식 결과 영상 생성
    dst = src.copy()
    for i in range(1, cnt):
        (x, y, w, h, s) = stats[i]
        if s < 1000:
            continue
        # 가장 유사한 숫자 이미지를 선택
        digit = find_digit(src_gray[y:y+h, x:x+w], img_digits)
        cv2.rectangle(dst, (x, y, x+w, y+h), (0, 255, 0), 2)
        cv2.putText(dst, str(digit), (x, y - 4), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

In [41]:
import sys
import numpy as np
import cv2

# 숫자 템플릿 이미지 0~9 로드
def load_digits():
    img_digits = []
    for i in range(10):
        filename = '../data/digits/digit{}.bmp'.format(i)
        img_digits.append(cv2.imread(filename, cv2.IMREAD_GRAYSCALE))
        if img_digits[i] is None:
            return None
    return img_digits

# 입력된 숫자 이미지와 가장 유사한 템플릿 인덱스 반환
def find_digit(img, img_digits):
    max_idx = -1
    max_ccoeff = -1
    img = cv2.resize(img, (100, 150))  # 템플릿 크기와 맞춤

    for i in range(10):
        res = cv2.matchTemplate(img, img_digits[i], cv2.TM_CCOEFF_NORMED)  # NCC 계산
        if res[0, 0] > max_ccoeff:
            max_idx = i
            max_ccoeff = res[0, 0]
    return max_idx

def main():
    # 입력 이미지 불러오기
    src = cv2.imread('../data/digits_print.bmp')
    if src is None:
        print('Image load failed!')
        return

    # 숫자 템플릿 이미지 불러오기
    img_digits = load_digits()
    if img_digits is None:
        print('Digit image load failed!')
        return

    # 그레이스케일 & 이진화
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    # 윤곽선(Contour) 추출
    contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 결과 영상 복사본
    dst = src.copy()

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)  # 바운딩 박스 계산
        if w * h < 1000:
            continue  # 너무 작은 영역은 무시

        digit_img = src_gray[y:y+h, x:x+w]  # 숫자 영역 잘라냄
        digit = find_digit(digit_img, img_digits)  # 템플릿 매칭으로 숫자 인식

        # 결과 시각화: 박스 + 숫자
        cv2.rectangle(dst, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(dst, str(digit), (x, y - 4), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # 결과 출력
    cv2.imshow('dst', dst)
    cv2.waitKey()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


### 케스케이드 검출
- Viola-Jones 얼굴 검출기
    - Positive 영상(얼굴)과 negative 영상(얼굴x)을 훈련하여 빠르고 정확한 얼굴 영역을 검출
    - 기존방법과 차이점? 
        - haar-like 특징을 사용
        - AdaBoost에 기반한 강한 분류 성능
        캐스케이드 방식을 통한 빠른 동작 속도.
    - 기존 얼굴 검출 방법보다 약 15배 빠르게 동작

### 유사 하르 haar-like features
- 사각형 형태의 필터 집합을 사용

### 케스케이드 검출기
- 일반적인 영상에는 얼굴이 한 두개 있을 뿐, 나머지 영역은 대부분 non-face 영역
- non-face영역을 빠르게 skip 하도록 다단계 검사 수행

In [46]:
import sys
import numpy as np
import cv2

src = cv2.imread('../data/lenna.bmp')

if src is None:
    print('Image load failed!')
    sys.exit()

classifier = cv2.CascadeClassifier('../data/haarcascade_frontalface_alt2.xml')

if classifier.empty():
    print('XML load failed!')
    sys.exit()
    
faces = classifier.detectMultiScale(src)

for (x, y, w, h) in faces:
    cv2.rectangle(src, (x, y, w, h), (255, 0, 255), 2)

cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()

In [49]:
import sys
import numpy as np
import cv2

# 이미지 불러오기
src = cv2.imread('../data/lenna.bmp')
if src is None:
    print('Image load failed!')
    sys.exit()

# 분류기 로드 (얼굴, 눈)
face_classifier = cv2.CascadeClassifier('../data/haarcascade_frontalface_alt2.xml')
eye_classifier = cv2.CascadeClassifier('../data/haarcascade_eye.xml')
if face_classifier.empty() or eye_classifier.empty():
    print('XML load failed!')
    sys.exit()

# 얼굴 검출
faces = face_classifier.detectMultiScale(src)
for (x1, y1, w1, h1) in faces:
    # 얼굴에 사각형 그리기
    cv2.rectangle(src, (x1, y1), (x1 + w1, y1 + h1), (255, 0, 255), 2)

    # 얼굴 상단 영역만 눈 검출용 ROI로 설정
    faceROI = src[y1:y1 + h1 // 2, x1:x1 + w1]

    # 눈 검출
    eyes = eye_classifier.detectMultiScale(faceROI)
    for (x2, y2, w2, h2) in eyes:
        center = (x2 + w2 // 2, y2 + h2 // 2)
        radius = w2 // 2
        cv2.circle(faceROI, center, radius, (255, 0, 0), 2, cv2.LINE_AA)

# 결과 출력
cv2.imshow('src', src)
cv2.waitKey()
cv2.destroyAllWindows()


### 영상의 흐름 인식

In [53]:
import sys
import random
import numpy as np
import cv2

# 동영상 불러오기
cap = cv2.VideoCapture('../data/vtest.mp4')
if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

# HOG 기술자 + 보행자용 SVM 디텍터 설정
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 매 프레임마다 보행자 검출
    detected, _ = hog.detectMultiScale(frame)

    # 검출된 영역마다 사각형 표시 (랜덤 색상)
    for (x, y, w, h) in detected:
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 3)

    # 결과 출력
    cv2.imshow('frame', frame)
    if cv2.waitKey(10) == 27:  # ESC 키 누르면 종료
        break

cap.release()
cv2.destroyAllWindows()